<a href="https://colab.research.google.com/github/JakeOh/202011_itw_bd21/blob/main/lab_ml/ml02_train_set_test_set.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
import pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.neighbors import KNeighborsClassifier  # 클래스
from sklearn.metrics import confusion_matrix, classification_report  # 함수, 속성
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 데이터 준비

Google Drive에 저장된 fish.pickle 파일에서 데이터를 로드.

In [2]:
file_path = '/content/drive/MyDrive/Colab Notebooks/lab_ml/fish.pickle'

In [3]:
with open(file_path, mode='rb') as f:
    fish = pickle.load(f)

In [4]:
type(fish)

dict

In [5]:
fish.keys()

dict_keys(['data', 'target'])

In [7]:
fish_data = fish['data']

In [8]:
fish_data.shape  
# row: 35 bream + 14 smelt
# column: Weigth + Length

(49, 2)

In [9]:
fish_data[:5]

array([[242. ,  25.4],
       [290. ,  26.3],
       [340. ,  26.5],
       [363. ,  29. ],
       [430. ,  29. ]])

In [10]:
fish_label = fish['target']

In [11]:
fish_label.shape

(49,)

In [12]:
fish_label[:5]

array([1, 1, 1, 1, 1])